In [1]:
import pickle
import time
import gc

In [2]:
max_size = 65
max_size = 39
print("max_size:", max_size)
num_try = 8
print("number of try:", num_try)

max_size: 39
number of try: 8


In [3]:
import mthree
from qiskit.test.mock import FakeManhattan
from libs_qrem import sgs_algorithm_py
from qiskit.ignis.mitigation.measurement import TensoredMeasFitter

In [4]:
def get_info(mit_hist, info):
    return {# "exact_one_norm_of_reduced_inv_A": mitigator.exact_one_norm_of_reduced_inv_A(),
            "mitigated_hist": mit_hist,
            "times": info["time"],
            "info": info,
            # "mitigation_stddev": mitigator.mitigation_stddev(norm_type = "exact"),
            }

In [5]:
def results_to_info(n):
    
    with open("pkls/results/results_" + str(n) + "-qubit.pkl", "rb") as f:
        results_list = pickle.load(f)
        f.close()
    
    mthree_lu_ghz_mqc_info_list = []
    for j in range(num_try):
        mthree_lu_ghz_mqc_info = {}
        mitigator = mthree.M3Mitigation(FakeManhattan())
        mitigator.single_qubit_cals = TensoredMeasFitter(results_list[j]["mit"], mit_pattern=[[k] for k in range(n)]).cal_matrices
        meas_layout = [k for k in range(n)]
        mit_hist, info = mitigator.apply_correction(results_list[j]["ghz"].get_counts(),
                                                    meas_layout, 
                                                    distance=None,
                                                    method='direct',
                                                    max_iter=25, 
                                                    tol=1e-5,
                                                    return_mitigation_overhead=True,
                                                    details=True)
        mthree_lu_ghz_mqc_info["ghz"] = get_info(sgs_algorithm_py(mit_hist), info)
        mthree_lu_ghz_mqc_info["mqc"] = []
        for i, mqc_hist in enumerate( results_list[j]["mqc"].get_counts() ):
            mit_hist, info = mitigator.apply_correction(mqc_hist,
                                                        meas_layout, 
                                                        distance=None,
                                                        method='direct',
                                                        max_iter=25, 
                                                        tol=1e-5,
                                                        return_mitigation_overhead=True,
                                                        details=True)
            mthree_lu_ghz_mqc_info["mqc"].append( get_info(sgs_algorithm_py(mit_hist), info) )
        mthree_lu_ghz_mqc_info_list.append(mthree_lu_ghz_mqc_info)
    with open("pkls/mthree_lu/info_list_" + str(n) + "-qubit.pkl", "wb") as f:
        pickle.dump(mthree_lu_ghz_mqc_info_list, f)
        f.close()

In [6]:
for n in range(1, max_size + 1):
    t1 = time.perf_counter()
    results_to_info(n)
    t2 = time.perf_counter()
    print(n, "qubit finished ( time:", t2 - t1, "s )")

1 qubit finished ( time: 0.06421151800000002 s )
2 qubit finished ( time: 0.13011862900000004 s )
3 qubit finished ( time: 0.08553816299999983 s )
4 qubit finished ( time: 0.15646028899999997 s )
5 qubit finished ( time: 0.2179844710000003 s )
6 qubit finished ( time: 0.205782283 s )
7 qubit finished ( time: 0.37037092599999966 s )
8 qubit finished ( time: 0.5153049299999997 s )
9 qubit finished ( time: 0.6903922119999999 s )
10 qubit finished ( time: 0.9667285059999999 s )
11 qubit finished ( time: 1.4930758380000002 s )
12 qubit finished ( time: 2.193777999000001 s )
13 qubit finished ( time: 2.8612715580000003 s )
14 qubit finished ( time: 5.281127078000003 s )
15 qubit finished ( time: 8.445232434999998 s )
16 qubit finished ( time: 12.396426098000003 s )
17 qubit finished ( time: 18.546183528 s )
18 qubit finished ( time: 29.354276181000003 s )
19 qubit finished ( time: 37.390668987 s )
20 qubit finished ( time: 48.548744298 s )
21 qubit finished ( time: 63.77779085 s )
22 qubit f

KeyboardInterrupt: 